<a href="https://colab.research.google.com/github/Shruthi973/HDS5210-02-Assignments/blob/main/midterm_2024final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [8]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculate PRIEST Clinical Severity Score.

    :param sex: Gender assigned at birth (str)
    :param age: Age in years (int)
    :param respiratory_rate: Respiratory rate in breaths per minute (int)
    :param oxygen_saturation: Oxygen saturation as a percent between 0 and 1 (float)
    :param heart_rate: Heart rate in beats per minute (int)
    :param systolic_bp: Systolic BP in mmHg (int)
    :param temperature: Temperature in degrees C (float)
    :param alertness: Alertness as a string description (str)
    :param inspired_oxygen: Inspired Oxygen as a string description (str)
    :param performance_status: Performance Status as a string description (str)
    :return: Risk percentage as a numeric value between 0 and 1 (float)

    >>> priest('male', 25, 20, 0.98, 70, 120, 37, 'alert', 'room air', 'unrestricted normal activity')
    0.01
    >>> priest('female', 75, 30, 0.9, 100, 90, 38.5, 'confused', 'supplemental oxygen', 'limited self care')
    0.59
    >>> priest('MALE', 50, 25, 0.95, 80, 110, 37.5, 'Alert', 'Room Air', 'Normal Activity')
    0.06
    """
    # Normalize input strings
    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    # Validate inputs
    if sex not in ['male', 'female'] or not (0 <= oxygen_saturation <= 1):
        return None

    score = 0

    # Sex
    if sex == 'male':
        score += 1

    # Age
    if 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4

    # Respiratory rate
    if 20 <= respiratory_rate <= 24:
        score += 1
    elif respiratory_rate >= 25:
        score += 2

    # Oxygen saturation
    if 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif 0.92 <= oxygen_saturation <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3

    # Heart rate
    if 91 <= heart_rate <= 110:
        score += 1
    elif 111 <= heart_rate <= 130:
        score += 2
    elif heart_rate > 130:
        score += 3

    # Systolic BP
    if 91 <= systolic_bp <= 100:
        score += 2
    elif systolic_bp <= 90:
        score += 3

    # Temperature
    if temperature >= 39:
        score += 2

    # Alertness
    if alertness in ['confused', 'voice', 'pain', 'unresponsive']:
        score += 3

    # Inspired oxygen
    if inspired_oxygen != 'room air':
        score += 2

    # Performance status
    if 'limited activity' in performance_status:
        score += 1
    elif 'limited self care' in performance_status:
        score += 2
    elif 'bed/chair bound' in performance_status:
        score += 3

    # Calculate probability
    if score <= 3:
        return 0.01
    elif 4 <= score <= 5:
        return 0.02
    elif 6 <= score <= 7:
        return 0.06
    elif 8 <= score <= 9:
        return 0.12
    elif 10 <= score <= 11:
        return 0.23
    elif 12 <= score <= 13:
        return 0.38
    elif 14 <= score <= 16:
        return 0.55
    elif 17 <= score <= 25:
        return 0.59
    else:
        return 0.99

# Run doctests for this function
import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)

Finding tests in NoName
Trying:
    priest('male', 25, 20, 0.98, 70, 120, 37, 'alert', 'room air', 'unrestricted normal activity')
Expecting:
    0.01
ok
Trying:
    priest('female', 75, 30, 0.9, 100, 90, 38.5, 'confused', 'supplemental oxygen', 'limited self care')
Expecting:
    0.59
ok
Trying:
    priest('MALE', 50, 25, 0.95, 80, 110, 37.5, 'Alert', 'Room Air', 'Normal Activity')
Expecting:
    0.06
ok


In [9]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 25, 20, 0.98, 70, 120, 37, 'alert', 'room air', 'unrestricted normal activity')
Expecting:
    0.01
ok
Trying:
    priest('female', 75, 30, 0.9, 100, 90, 38.5, 'confused', 'supplemental oxygen', 'limited self care')
Expecting:
    0.59
ok
Trying:
    priest('MALE', 50, 25, 0.95, 80, 110, 37.5, 'Alert', 'Room Air', 'Normal Activity')
Expecting:
    0.06
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [10]:
import requests

def find_hospital(age, sex, risk):
    """
    Find the appropriate hospital based on age, sex, and risk.

    :param age: Age in years (int)
    :param sex: Gender (str)
    :param risk: Risk percentage as a float between 0 and 1
    :return: Hospital name (str) or None if request fails

    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(25, 'female', 0.05)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(70, 'male', 0.8)
    'Wesley Woods Geriatric Hospital'
    """
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex.lower()}&risk_pct={risk}"
    try:
        response = requests.get(url)
        data = response.json()
        return data['hospital']
    except:
        return None

# Run doctests for this function
import doctest
doctest.run_docstring_examples(find_hospital, globals(), verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(25, 'female', 0.05)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(70, 'male', 0.8)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


In [11]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(25, 'female', 0.05)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(70, 'male', 0.8)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [12]:
import json
import requests

def get_address(hospital_name):
    """
    Get the address of a hospital from a JSON file.

    :param hospital_name: Name of the hospital (str)
    :return: Address of the hospital (str) or None if not found

    >>> get_address("GRADY MEMORIAL HOSPITAL")
    '80 JESSE HILL, JR DRIVE SE'

    >>> get_address("EMORY UNIVERSITY HOSPITAL MIDTOWN")
    '550 PEACHTREE ST NE'

    >>> get_address("PIEDMONT HOSPITAL")
    '1968 PEACHTREE ROAD NW'

    >>> get_address("NONEXISTENT HOSPITAL") is None
    True

    >>> get_address("") is None
    True
    """
    if not isinstance(hospital_name, str) or not hospital_name:
        return None

    try:
        with open('ga_hospitals (3).json', 'r') as f:
            hospitals = json.load(f)

        if hospital_name in hospitals:
            return hospitals[hospital_name]["ADDRESS"]
        return None

    except (FileNotFoundError, json.JSONDecodeError):
        return None
import doctest
doctest.run_docstring_examples(get_address, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_address("GRADY MEMORIAL HOSPITAL")
Expecting:
    '80 JESSE HILL, JR DRIVE SE'
**********************************************************************
File "__main__", line 11, in NoName
Failed example:
    get_address("GRADY MEMORIAL HOSPITAL")
Expected:
    '80 JESSE HILL, JR DRIVE SE'
Got nothing
Trying:
    get_address("EMORY UNIVERSITY HOSPITAL MIDTOWN")
Expecting:
    '550 PEACHTREE ST NE'
**********************************************************************
File "__main__", line 14, in NoName
Failed example:
    get_address("EMORY UNIVERSITY HOSPITAL MIDTOWN")
Expected:
    '550 PEACHTREE ST NE'
Got nothing
Trying:
    get_address("PIEDMONT HOSPITAL")
Expecting:
    '1968 PEACHTREE ROAD NW'
**********************************************************************
File "__main__", line 17, in NoName
Failed example:
    get_address("PIEDMONT HOSPITAL")
Expected:
    '1968 PEACHTREE ROAD NW'
Got nothing
Trying:
    get_address("NONEXISTENT HOSPI

## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [13]:
import requests

def process_people(file_url):
    """
    Process people data from a PSV file and return a list of dictionaries with patient information.

    :param file_url: URL of the PSV file
    :return: List of dictionaries with patient information
    """
    result = []
    try:
        response = requests.get(file_url)
        lines = response.text.strip().split('\n')
        headers = lines[0].split('|')
        for line in lines[1:]:
            data = line.split('|')
            patient = dict(zip(headers, data))
            risk = priest(patient['Sex'], int(patient['Age']), int(patient['RespiratoryRate']),
                          float(patient['O2Saturation']), int(patient['HeartRate']),
                          int(patient['SystolicBP']), float(patient['Temperature']),
                          patient['Alertness'], patient['InspiredOxygen'],
                          patient['PerformanceStatus'])
            if risk is not None:
                hospital = find_hospital(int(patient['Age']), patient['Sex'], risk)
                address = get_address(hospital)
                result.append({
                    patient['PatientNumber']: [
                        patient['Sex'], patient['Age'], patient['RespiratoryRate'],
                        patient['O2Saturation'], patient['HeartRate'], patient['SystolicBP'],
                        patient['Temperature'], patient['Alertness'], patient['InspiredOxygen'],
                        patient['PerformanceStatus'], hospital, address
                    ]
                })
    except:
        pass
    return result

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [14]:
import json
import requests

def check_results(processed_data):
    url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
    try:
        response = requests.get(url)
        expected_results = json.loads(response.text)

        if len(processed_data) != len(expected_results):
            print(f"Mismatch in number of records. Expected: {len(expected_results)}, Got: {len(processed_data)}")
            return

        for processed, expected in zip(processed_data, expected_results):
            if processed != expected:
                print(f"Mismatch in data: \nProcessed: {processed}\nExpected: {expected}")
                return

        print("All results match the expected output!")
    except:
        print("Error occurred while checking results")

# Run the entire process
if __name__ == "__main__":
    file_url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
    processed_data = process_people(file_url)
    check_results(processed_data)

# Run doctests for all functions
import doctest
doctest.testmod(verbose=True)

Mismatch in number of records. Expected: 99, Got: 0
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(25, 'female', 0.05)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(70, 'male', 0.8)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    get_address("GRADY MEMORIAL HOSPITAL")
Expecting:
    '80 JESSE HILL, JR DRIVE SE'
**********************************************************************
File "__main__", line 11, in __main__.get_address
Failed example:
    get_address("GRADY MEMORIAL HOSPITAL")
Expected:
    '80 JESSE HILL, JR DRIVE SE'
Got nothing
Trying:
    get_address("EMORY UNIVERSITY HOSPITAL MIDTOWN")
Expecting:
    '550 PEACHTREE ST NE'
**********************************************************************
File "__main__", line 14, in __main__.get_address
Failed example:
    get_address("EMORY UNIVERSITY HOSPITAL MIDTOWN")
Expected:
    '550 PEA

TestResults(failed=3, attempted=11)

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---